### Notebook used to replicate the LeNet-5 model used in handwriting analysis

##### The original publication is: Lecun, Y., Bottou, L., Bengio, Y., & Haffner, P. (1998). Gradient-based learning applied to document recognition. Proceedings of the IEEE, 86(11), 2278–2324. doi:10.1109/5.726791 

##### The first few replications will follow tutorials: https://medium.datadriveninvestor.com/architecture-implementation-of-lenet-from-scratch-in-pytorch-709cc38c00a9

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchinfo import summary

import time

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6, 
                                kernel_size = 5, stride = 1, padding = 0)
        self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, 
                                kernel_size = 5, stride = 1, padding = 0)
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 120, 
                                kernel_size = 5, stride = 1, padding = 0)
        self.linear1 = nn.Linear(120, 84)
        self.linear2 = nn.Linear(84, 10)
        self.tanh = nn.Tanh()
        self.avgpool = nn.AvgPool2d(kernel_size = 2, stride = 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.tanh(x)
        x = self.avgpool(x)
        x = self.conv2(x)
        x = self.tanh(x)
        x = self.avgpool(x)
        x = self.conv3(x)
        x = self.tanh(x)

        x = x.reshape(x.shape[0], -1)
        x = self.linear1(x)
        x = self.tanh(x)
        x = self.linear2(x)
        return x

model = LeNet()
x = torch.randn(64,1,32,32)
output = model(x)

print(model)
print("output.shape : ",output.shape)
summary(model, input_size=(64,1,32,32))

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (linear1): Linear(in_features=120, out_features=84, bias=True)
  (linear2): Linear(in_features=84, out_features=10, bias=True)
  (tanh): Tanh()
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
)
output.shape :  torch.Size([64, 10])


Layer (type:depth-idx)                   Output Shape              Param #
LeNet                                    [64, 10]                  --
├─Conv2d: 1-1                            [64, 6, 28, 28]           156
├─Tanh: 1-2                              [64, 6, 28, 28]           --
├─AvgPool2d: 1-3                         [64, 6, 14, 14]           --
├─Conv2d: 1-4                            [64, 16, 10, 10]          2,416
├─Tanh: 1-5                              [64, 16, 10, 10]          --
├─AvgPool2d: 1-6                         [64, 16, 5, 5]            --
├─Conv2d: 1-7                            [64, 120, 1, 1]           48,120
├─Tanh: 1-8                              [64, 120, 1, 1]           --
├─Linear: 1-9                            [64, 84]                  10,164
├─Tanh: 1-10                             [64, 84]                  --
├─Linear: 1-11                           [64, 10]                  850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
To

In [4]:
# Hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
learning_rate = 0.01
num_epochs = 10

train_dataset = datasets.MNIST(root='dataset/', train=True, 
                               transform=transforms.Compose([transforms.Pad(2), transforms.ToTensor()]), download=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, 
                              transform=transforms.Compose([transforms.Pad(2), transforms.ToTensor()]), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)
dataset_sizes = {'train':len(train_dataset), 'test':len(test_dataset)}

model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

from IPython.display import HTML, display
class ProgressMonitor(object):
    """
    Custom IPython progress bar for training
    """
    
    tmpl = """
        <p>Loss: {loss:0.4f}   {value} / {length}</p>
        <progress value='{value}' max='{length}', style='width: 100%'>{value}</progress>
    """
 
    def __init__(self, length):
        self.length = length
        self.count = 0
        self.display = display(self.html(0, 0), display_id=True)
        
    def html(self, count, loss):
        return HTML(self.tmpl.format(length=self.length, value=count, loss=loss))
        
    def update(self, count, loss):
        self.count += count
        self.display.update(self.html(self.count, loss))

def train_new(model,criterion,optimizer,num_epochs,dataloaders,dataset_sizes,first_epoch=1):
    since = time.time() 
    best_loss = 999999
    best_epoch = -1
    last_train_loss = -1
    plot_train_loss = []
    plot_valid_loss = []


    for epoch in range(first_epoch, first_epoch + num_epochs):
        print("")
        print('Epoch', epoch)
        running_loss = 0.0
        valid_loss = 0.0

        # train phase
        model.train()

        # create a progress bar
        progress = ProgressMonitor(length=dataset_sizes["train"])

        for data in dataloaders[0]:
            # Move the training data to the GPU
            inputs, labels  = data
            batch_size = inputs.shape[0]

            inputs = Variable(inputs.to(device))
            labels = Variable(labels.to(device))

            # clear previous gradient computation
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.data * batch_size
            # update progress bar
            progress.update(batch_size, running_loss)

    epoch_loss = running_loss / dataset_sizes["train"]
    print('Training loss:', epoch_loss.item())
    plot_train_loss.append(epoch_loss)

    # validation phase
    model.eval()
    # We don't need gradients for validation, so wrap in 
    # no_grad to save memory
    with torch.no_grad():
        for data in dataloaders[-1]:
            inputs, labels  = data
            batch_size = inputs.shape[0]

            inputs = Variable(inputs.to(device))
            labels = Variable(labels.to(device))
            outputs = model(inputs)

            # calculate the loss
            optimizer.zero_grad()
            loss = criterion(outputs, labels)

            # update running loss value
            valid_loss += loss.data * batch_size

    epoch_valid_loss = valid_loss / dataset_sizes["test"]
    print('Validation loss:', epoch_valid_loss.item())
    plot_valid_loss.append(epoch_valid_loss)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return plot_train_loss, plot_valid_loss, model

if __name__=="__main__":
    train_losses, valid_losses, model = train_new(
        model = model, criterion = criterion, optimizer = optimizer,
        num_epochs=10,dataloaders = [train_loader, test_loader],
        dataset_sizes = dataset_sizes)

0.3%

100.0%


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


100.0%


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



...


Epoch 7



Epoch 8



Epoch 9



Epoch 10


Training loss: 0.12198857963085175
Validation loss: 0.19728846848011017
Training complete in 1m 39s


In [5]:
def accuracy(loader, model, train=True):
    num_correct = num_samples = 0
    model.eval()
    with torch.no_grad():
      for data in loader:
        inputs, labels  = data
        batch_size = inputs.shape[0]
        
        inputs = Variable(inputs.to(device))
        labels = Variable(labels.to(device))
 
        outputs = model(inputs)
        _, preds = outputs.max(1)
        num_correct += (preds == labels).sum()
        num_samples += preds.size(0)
    accuracy = (num_correct.item()/num_samples)*100
    if train:
      print("Model Predicted {} correctly out of {} from training dataset, Acuracy : {:.2f}".format(num_correct.item(), num_samples, accuracy))
    else:
      print("Model Predicted {} correctly out of {} from testing dataset, Acuracy : {:.2f}".format(num_correct.item(), num_samples, accuracy))
    model.train()

accuracy(train_loader, model)
accuracy(test_loader, model, train=False)

Model Predicted 56673 correctly out of 60000 from training dataset, Acuracy : 94.45
Model Predicted 9428 correctly out of 10000 from testing dataset, Acuracy : 94.28
